<a href="https://colab.research.google.com/github/Yyssjj96/__practice___/blob/main/a_b_test_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.3 MB/s eta 0:00:00


In [ ]:
from faker import Faker
import random
import pandas as pd

fake = Faker()

# 데이터 생성 함수 정의
def generate_dummy_data(num_users, with_advertisement):
    data = []
    for _ in range(num_users):
        user_id = fake.random_int(min=10000, max=99999)
        has_advertisement = with_advertisement
        retention_7 = random.choice([True, False])  # 재방문 여부 (7일 이후)

        data.append([user_id, has_advertisement, retention_7])

    return pd.DataFrame(data, columns=['userid', 'has_advertisement', 'retention_7'])

# 광고가 없는 그룹 데이터 생성 (300명)
group_no_ad = generate_dummy_data(num_users=300, with_advertisement=False)

# 광고가 있는 그룹 데이터 생성 (300명)
group_with_ad = generate_dummy_data(num_users=300, with_advertisement=True)

# 두 데이터프레임 합치기
dummy_data = pd.concat([group_no_ad, group_with_ad], ignore_index=True)

# 데이터 저장
dummy_data.to_csv('dummy_ab_test_data.csv', index=False)

In [ ]:
import pandas as pd
from scipy import stats

df = pd.read_csv('dummy_ab_test_data.csv')

print(df.head())

# 광고가 있는 그룹과 없는 그룹으로 데이터 분할
group_no_ad = df[df['has_advertisement'] == False]['retention_7']
group_with_ad = df[df['has_advertisement'] == True]['retention_7']

# 각 그룹의 샘플 크기 확인
print(f'광고가 없는 그룹 샘플 수: {len(group_no_ad)}')
print(f'광고가 있는 그룹 샘플 수: {len(group_with_ad)}')

   userid  has_advertisement  retention_7
0   53974              False        False
1   78371              False         True
2   68149              False        False
3   92869              False         True
4   40320              False        False
광고가 없는 그룹 샘플 수: 300
광고가 있는 그룹 샘플 수: 300


In [ ]:
# 각 그룹의 평균 재방문율 계산
mean_no_ad = group_no_ad.mean()
mean_with_ad = group_with_ad.mean()

print(f'광고가 없는 그룹 재방문율: {mean_no_ad:.2%}')
print(f'광고가 있는 그룹 재방문율: {mean_with_ad:.2%}')

# 두 그룹의 평균 차이 계산
mean_diff = mean_with_ad - mean_no_ad
print(f'평균 차이: {mean_diff:.2%}')

# t-검정 수행
t_stat, p_value = stats.ttest_ind(group_no_ad, group_with_ad)
print(f't-검정 통계량: {t_stat:.2f}')
print(f'p-value: {p_value:.4f}')

# p-value 기준으로 유의수준 설정
alpha = 0.05
if p_value < alpha:
    print('통계적으로 유의미한 차이가 있음')
else:
    print('통계적으로 유의미한 차이가 없음')

광고가 없는 그룹 재방문율: 52.67%
광고가 있는 그룹 재방문율: 45.33%
평균 차이: -7.33%
t-검정 통계량: 1.80
p-value: 0.0726
통계적으로 유의미한 차이가 없음
